# CSC446 Final Project 

The program below simulates a subsection of Big White ski-lift network, specifically the parallel Snow Ghost Express and Ridge Rocket Express which feed into the Alpine T-bar. The 3 arrival rates (x value) and 3 average delays (y value) have been combined to test all possible combinations, each combination run with 5 different random seeds and with n = 10,000 and n = 100,000. Each combination produces a group of statistics which are averaged across the 5 different seeds. 


**SIMULATION PARAMETERS**

|  Q1 Service Time  | Q2 Service Time   | Q3 Service Time | Q1 Arrival Rate | Q2 Arrival Rate | Q3 Arrival Rate | Average Delay | Random Seed |
|:------------------------| :------------------: |:------------------------| :------------------------| :------------------------| :------------------------| :------------------------|:------------------------|
| 3000/hour  | 2800/hour |  600/hour | Mean interarrival 30s| Mean interarrival 25s | Mean interarrival 55s | |
| 3000/hour  | 2800/hour |  600/hour | Mean interarrival 30s| Mean interarrival 25s | Mean interarrival 50s | |
| 3000/hour  | 2800/hour |  600/hour | Mean interarrival 30s| Mean interarrival 25s | Mean interarrival 45s | |
| 3000/hour  | 2800/hour |  600/hour | Mean interarrival 30s| Mean interarrival 30s | Mean interarrival 55s | |
| 3000/hour  | 2800/hour |  600/hour | Mean interarrival 30s| Mean interarrival 30s | Mean interarrival 50s | |
| 3000/hour  | 2800/hour |  600/hour | Mean interarrival 30s| Mean interarrival 30s | Mean interarrival 45s | |
| 3000/hour  | 2800/hour |  600/hour | Mean interarrival 25s| Mean interarrival 25s | Mean interarrival 55s | |
| 3000/hour  | 2800/hour |  600/hour | Mean interarrival 25s| Mean interarrival 25s | Mean interarrival 50s | |
| 3000/hour  | 2800/hour |  600/hour | Mean interarrival 25s| Mean interarrival 25s | Mean interarrival 45s | |
| 3000/hour  | 2800/hour |  600/hour | Mean interarrival 30s| Mean interarrival 25s | Mean interarrival 55s | |
| 3000/hour  | 2800/hour |  600/hour | Mean interarrival 30s| Mean interarrival 25s | Mean interarrival 50s | |
| 3000/hour  | 2800/hour |  600/hour | Mean interarrival 30s| Mean interarrival 25s | Mean interarrival 45s | |


In [ ]:

import numpy as np
np.random.seed(seed_n)

class SimpleQueue:
    """This simulation replicates a subsection of the Big White ski resort and tests various arrival rates"""

    def __init__(self, num_delays_required):
        """Initialization function"""

        # Specify input parameters.
        
        # Service times 
        # Given in service times per customer 
        self.mean_service_a = 3000/60  # 50 customers per minute 
        self.mean_service_b = 2800/60 # 46.67 customers per minute 
        self.mean_service_c =  600/60 # 10 customers per minute 
        self.num_delays_required = num_delays_required

        # Initialize state variables.
        self.server_status_a = 0  # 0 is idle and 1 is busy.
        self.server_status_b = 0 
        self.server_status_c = 0 
        self.num_in_qa = 0
        self.num_in_qb = 0
        self.num_in_qc = 0 
        self.time_arrival_a = []  # List for times of arrival of customers.
        self.time_arrival_b = [] 
        self.time_arrival_c = [] 
        self.time_last_event = 0.0

        # Initialize statistical counters.
        self.num_custs_delayed_a = 0
        self.num_custs_delayed_b = 0
        self.num_custs_delayed_c = 0 
        self.total_delay_a = 0.0
        self.total_delay_b = 0.0
        self.total_delay_c = 0.0 
        self.area_num_in_qa = 0.0
        self.area_num_in_qb = 0.0
        self.area_num_in_qc = 0.0 
        self.area_server_status_a = 0.0
        self.area_server_status_b = 0.0 
        self.area_server_status_c = 0.0
        
        # Create lists for time of each path 
        self.time_in_system_a = [] 
        self.time_in_system_b = []
        
        # Use dictionary to track customer entry times 
        self.customer_entry_times = {} 
        self.customer_id = 0 
        ## THINK ABOUT HOW TO DEAL W THIS IN DELAYS 
        self.group_size = 0

        # Initialize simulation clock.
        self.sim_time = 0.0

        # Initialize event list.
        self.time_next_event = [0, 0, 0, 0, 0, 0]
        self.time_next_event[1] = self.sim_time + self.expon(self.interarrival_a)
        self.time_next_event[2] = float('inf')
        # Arrival and departures for Server B 
        self.time_next_event[3] = self.sim_time + self.expon(self.interarrival_b)
        self.time_next_event[4] = float('inf')
        # Arrival and departures for Server C 
        self.time_next_event[5] = float('inf')

        # Initialize other variables.
        self.num_events = 5

    def main(self, int_a, int_b, seed_n, run):
        # Interarrival times for the two Express lifts 
        self.interarrival_a = int_a # minutes 
        self.interarrival_b = int_b # minutes 
        
        # Run the simulation while more delays are needed.
        while self.num_custs_delayed_c < self.num_delays_required:

            self.timing()

            # Update the time-average statistical counters.
            self.update_time_avg_stats()

            # Invoke the appropriate event function.
            if self.next_event_type == 1:
                self.arrive_a()
            elif self.next_event_type == 2:
                self.depart_a()
            elif self.next_event_type == 3:
                self.arrive_b()
            elif self.next_event_type == 4:
                self.depart_b() 
            elif self.next_event_type == 5:
                self.depart_c()
            else:
                break

        # Invoke the report generator.
        self.report()

    def timing(self):
        """Timing function."""

        # Initialize variables needed to search event list for minimum time.
        self.min_time_next_event = float('inf')
        self.next_event_type = 0

        # Determine the event type of the next event to occur.
        for i in range(1, self.num_events+1):
            if self.time_next_event[i] < self.min_time_next_event:
                self.min_time_next_event = self.time_next_event[i]
                self.next_event_type = i

        # Check to see whether all entries in the event list have 'infinite' values.
        if self.next_event_type == 0:
            # Stop the simulation.
            raise Exception("All entries in the event list have infinite values, so stop the simulation.")

        # The event list has an entry with a finite value, so advance the simulation clock.
        self.sim_time = self.min_time_next_event

    def arrive_a(self):
        # This is the arrival function for the Snow Ghost Express 
        # If the line is too long (ie: server is busy),
        # the customer goes to the Ridge Rocket Express 

        # Utilize dictionary to record entry and id 
        id = self.customer_id 
        # Each arrival is a group between 1 and 6 customers 
        self.group_size = np.random.randit(1,7)
        self.customer_id += 1
        self.customer_entry_times[id] = self.sim_time
        
        # Schedule next arrival.
        self.time_next_event[1] = self.sim_time + self.expon(self.interarrival_a)

        # Check to see whether the server is busy.
        if self.server_status_a == 1:
            # Server is busy, so increment the number of customers in queue
            # Need to increment the number in queue by the group size 
            self.num_in_qa += self.group_size 
            self.time_arrival_a.append([self.sim_time, id, self.group_size])
        else:
            # Server is idle, so arriving customer has a delay of zero.
            self.delay = 0.0
            self.total_delay_a += self.delay

            # Increment the number of customers delayed and make server busy.
            self.num_custs_delayed_a += self.group_size
            self.server_status_a = 1

            # Schedule a departure.
            self.time_next_event[2] = self.sim_time + self.expon(self.mean_service_a)
            self.current_cust_a = id 
            return self.group_size

    def depart_a(self):
        """Depart event function."""
        
        # Get customer that just finished
        if hasattr(self, 'current_cust_a'):
            id = self.current_cust_a
        else:
            id = None
            
        # Check to see whether the queue is empty.
        if self.num_in_qa == 0:
            # The queue is empty, so make the server idle
            self.server_status_a = 0
            self.time_next_event[2] = float('inf')
        else:
            # The queue is nonempty, so decrement the number in queue.
            self.num_in_qa -= group_size 
            arrival_time, id = self.time_arrival_a.pop(0)
            
            self.delay = self.sim_time - arrival_time
            self.total_delay_a += self.delay

            # Increment the number of customers delayed and schedule a departure.
            self.num_custs_delayed_a += group_size 
            self.time_next_event[2] = self.sim_time + self.expon(self.mean_service_a)
            self.current_cust_a = id

        # Send customer to Queue C 
        if id is not None: 
            self.arrive_c('A', id) 
            
    def arrive_b(self):
        # This is the arrival function for the Ridge Rocket Express 
        # If the line is too long (ie: server is busy),
        # the customer goes to the Snow Ghost Express 
        
        # Utilize dictionary to record entry and id 
        id = self.customer_id 
        # Each arrival is a group between 1 and 6 customers 
        group_size = np.random.randit(1,7)
        self.customer_id += 1
        self.customer_entry_times[id] = self.sim_time
        
        # Schedule next arrival.
        self.time_next_event[3] = self.sim_time + self.expon(self.interarrival_b)

        # Check to see whether the server is busy.
        if self.server_status_b == 1:
            # Server is busy, so increment the number of customers in queue
            self.num_in_qb += self.group_size 
            self.time_arrival_b.append([self.sim_time, id])
        else:
            # Server is idle, so arriving customer has a delay of zero
            self.delay = 0.0
            self.total_delay_b += self.delay

            # Increment the number of customers delayed and make server busy
            self.num_custs_delayed_b += self.group_size
            self.server_status_b = 1

            # Schedule a departure 
            self.time_next_event[4] = self.sim_time + self.expon(self.mean_service_b)
            self.current_cust_b = id 

    def depart_b(self):
        # The 6 customers depart and go to the T-bar queue 
        # Get customer that just finished
        if hasattr(self, 'current_cust_b'):
            id = self.current_cust_b
        else:
            id = None
            
        # Check to see whether the queue is empty.
        if self.num_in_qb == 0:
            # The queue is empty, so make the server idle
            self.server_status_b = 0
            self.time_next_event[4] = float('inf')
        else:
            # The queue is nonempty, so decrement the number in queue.
            self.num_in_qb -= self.group_size
            arrival_time, id = self.time_arrival_b.pop(0)

            # Compute the delay of the customer who is beginning service
            self.delay = self.sim_time - arrival_time
            self.total_delay_b += self.delay

            # Increment the number of customers delayed and schedule a departure.
            self.num_custs_delayed_b += self.group_size
            self.time_next_event[4] = self.sim_time + self.expon(self.mean_service_b)
            self.current_cust_b = id 

        # Send customer to Queue C (for BOTH empty and non-empty queue cases)
        if id is not None:
            self.arrive_c('B', id)
            
    def arrive_c(self, origin, id):
        # This is the arrival function for the Alpine T-bar 
        # Customers arrive here from the Snow Ghost Express and Ridge Rocket Express 

        # Check to see whether the server is busy.
        if self.server_status_c == 1:
            # If the line is too long, the group will balk (leave)
            if _______: 
                # Server is busy, so increment the number of customers in queue
                self.num_in_qc += self.group_size
                self.time_arrival_c.append([self.sim_time, origin, id])
        else:
            # Server is idle, so arriving customer has a delay of zero.
            self.delay = 0.0
            self.total_delay_c += self.delay

            # Make server busy.
            self.server_status_c = 1

            # Schedule a departure.
            self.time_next_event[5] = self.sim_time + self.expon(self.mean_service_c)
            self.current_cust_c = (origin, id)

    def depart_c(self):
        """Depart event function."""
        if hasattr(self, 'current_cust_c'):
            origin, id = self.current_cust_c
            
            # Use id to calc total time in system for customer 
            if id in self.customer_entry_times:
                total_time = self.sim_time - self.customer_entry_times[id]
                if origin == 'A':
                    self.time_in_system_a.append(total_time)
                else:
                    self.time_in_system_b.append(total_time)
                del self.customer_entry_times[id]
                
        # Check to see whether the queue is empty.
        if self.num_in_qc == 0:
            # The queue is empty, so make the server idle
            self.server_status_c = 0
            self.time_next_event[5] = float('inf')
        else:
            # The queue is nonempty, so decrement the number in queue.
            self.num_in_qc -= self.group_size
            arrival_time, origin, id = self.time_arrival_c.pop(0)

            # Compute the delay of the customer who is beginning service
            self.delay = self.sim_time - arrival_time
            self.total_delay_c += self.delay

            # Increment the number of customers delayed and schedule a departure.
            self.num_custs_delayed_c += self.group_size
            self.time_next_event[5] = self.sim_time + self.expon(self.mean_service_c)
            self.current_cust_c = (origin, id)

    def report(self):
        """Report generator function."""
        time_in_system_total = self.time_in_system_a + self.time_in_system_b
        print(f"Run number: {run}, Seed: {seed_n}")
        print("Average total time in the system: ", f"{np.mean(time_in_system_total):.3f}")
        print("Average total time spent in the system by Q1 passengers: ", f"{np.mean(self.time_in_system_a):.3f}")
        print("Average total time spent in the system by Q2 passengers: ", f"{np.mean(self.time_in_system_b):.3f}")
        ave_num_a = (1/self.interarrival_a) * np.mean(self.time_in_system_a) 
        print("Average total number of Q1 passengers in the system: ", f"{(ave_num_a):.3f}")
        ave_num_b = x(1/self.interarrival_b) * np.mean(self.time_in_system_b) 
        print("Average total number of Q2 passengers in the system: ", f"{(ave_num_b):.3f}")
        print("-" * 40)
        
    def update_time_avg_stats(self):
        """Function to compute time-average statistics."""

        # Compute time since last event and update time-of-last-event marker.
        self.time_since_last_event = self.sim_time - self.time_last_event
        self.time_last_event = self.sim_time

        # Update area under number-in-queue variable.
        self.area_num_in_qc += self.num_in_qc * self.time_since_last_event
        self.area_num_in_qa += self.num_in_qa * self.time_since_last_event 
        self.area_num_in_qb += self.num_in_qb * self.time_since_last_event 
        # Update area under the server-busy indicator variable.
        self.area_server_status_a += self.server_status_a * self.time_since_last_event
        self.area_server_status_b += self.server_status_b * self.time_since_last_event
        self.area_server_status_c += self.server_status_c * self.time_since_last_event

    def expon(self, mean):
        """Function to generate exponential random variates."""
        return -mean * np.log(np.random.uniform(0, 1))

# Run SimpleQueue with m = 10,000 and 100,000 
m = SimpleQueue(100000)

# Call main function to start the simulation.
# Reconfigure main function to accept 
# interarrival q1, interarrival q2 
# average delay 
m.main(int_a, int_b, run, seed_n)
# input variables will be int_a, int_b, y, seed_n, run 

NameError: name 'run' is not defined